In [ ]:
import gym
import numpy as np
from tqdm import tqdm
import torch
import wandb

from fractal_zero.config import FMCConfig
from fractal_zero.search.fmc import FMC
from fractal_zero.models.prediction import FullyConnectedPredictionModel
from fractal_zero.vectorized_environment import (
    RayVectorizedEnvironment,
    VectorizedDynamicsModelEnvironment,

)
from fractal_zero.trainers.online import OnlineFMCPolicyTrainer

from fractal_zero.tests.test_vectorized_environment import build_test_joint_model

In [ ]:
NUM_WALKERS = 64

class CartpolePolicy(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.net = torch.nn.Sequential(
            torch.nn.Linear(4, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 1),
        )

    def forward(self, observations, with_randomness: bool = False):
        observations = torch.tensor(observations).float()

        y = self.net(observations)

        if with_randomness:
            # center = embeddings.std()
            center = y.var()
            centered_uniform_noise = (torch.rand_like(y) * center) - (center / 2)
            y += centered_uniform_noise

        return y

    def parse_actions(self, actions):
        actions = torch.where(actions > 0.5, 1, 0).flatten()
        l = actions.tolist()
        if len(l) == 1:
            return l[0]

policy_model = CartpolePolicy()

In [ ]:
# optimizer = torch.optim.SGD(policy_model.parameters(), lr=0.01, weight_decay=1e-4)
optimizer = torch.optim.Adam(policy_model.parameters(), lr=0.03, weight_decay=1e-3)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)
policy_trainer = OnlineFMCPolicyTrainer("CartPole-v0", policy_model, optimizer, NUM_WALKERS)

In [ ]:
wandb.init(project="fz-policy-trainer-game-tree")

In [ ]:
num_episodes = 100
train_steps_per_episode = 100
eval_every = 20
max_steps = 200

for _ in range(num_episodes):
    policy_trainer.generate_episode_data(max_steps)

    for i in range(train_steps_per_episode):
        policy_trainer.train_on_latest_episode()

        if i % eval_every == 0:
            policy_trainer.evaluate_policy(max_steps)

    # lr_scheduler.step()

In [ ]:
policy_trainer.fmc.clone_receives

In [ ]:
# policy_trainer.fmc.tree.render()

In [ ]:
str(policy_trainer.fmc.tree.g)

In [ ]:
# tree = policy_trainer.fmc.tree.g
# for node in tree:
#     print(node, tree.out_degree(node), node.visits)